In [1]:
import os 
import sys
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from scipy.io import loadmat


import pyod packages and methods

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging


C:\Users\hp\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


import performance metrics package

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score


define data file and read X and y

In [4]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat',
              'letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat'
              ,'pendigits.mat','pima.mat','satellite.mat','satimage-2.mat',
              'shuttle.mat','vertebral.mat','vowels.mat','wbc.matb']
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.matb']

In [5]:
data=loadmat('cardio.mat')
data 


{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [6]:
len(data)

5

In [7]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [8]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [9]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

In [10]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

define nine outliers to be compared 

In [11]:
df_columns=['Data','#Samples','#Dimeensions','Outlier Prec',
           'ABOD','CBLOF','FB','IForest','KNN','LOF','MCD',
           'OCSVM','PCA']


In [12]:
roc_df = pd.DataFrame(columns= df_columns)
roc_df

,Data,#Samples,#Dimeensions,Outlier Prec,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA


PRECISION_N_SCORES -Performance evaluation table 

In [13]:
prn_df =pd.DataFrame(columns=df_columns)
prn_df

,Data,#Samples,#Dimeensions,Outlier Prec,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA


TIME DATAFRAME

In [14]:
time_df =pd.DataFrame(columns=df_columns)
time_df

,Data,#Samples,#Dimeensions,Outlier Prec,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA


EXPLORING ALL MAT FILES

In [ ]:
#Exploring/Accessing all mat files
from time import time
random_state = np.random.RandomState(42)


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, "...")
    mat = loadmat(os.path.join(mat_file))
    
    X = mat["X"]
    y = mat["y"].ravel()
    
    outliers_fraction = np.count_nonzero(y)/len(y)
    outliers_percentage = round(outliers_fraction*100, ndigits=4)
    
    #construct containers for saving results
    roc_list = [mat_file[:-4],X.shape[0],X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4],X.shape[0],X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4],X.shape[0],X.shape[1], outliers_percentage]
    
    #60% data for training and 40% for testing
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state = 2)
    
    # STandardaizing data for preprocessing
    X_train_norm,  X_test_norm = standardizer(X_train,X_test)
    
    classifiers = {
                   "Angle based outlier detector(ABOD)":ABOD(contamination = outliers_fraction), #contamination based on what factors the data is considered as outliers
                   "Cluster-based Loacl Outlier Factor":CBLOF(contamination = outliers_fraction,check_estimator= False, random_state=random_state),
                   "Feature Bagging":FeatureBagging(contamination = outliers_fraction,random_state=random_state),
                   "Histogram base outlier detection": HBOS(contamination = outliers_fraction),
                   "Isolation Forest": IForest(contamination = outliers_fraction,random_state=random_state),
                   "k Nearest Neighbours (kNN)": KNN(contamination = outliers_fraction),
                   "Local Outlier Factor(LOF)": LOF(contamination = outliers_fraction),
                   "Minimun Covariance Determinant (MCD)": MCD(contamination = outliers_fraction, random_state=random_state),
                   "One-Class SVM (OCSVM)": OCSVM(contamination = outliers_fraction),
                   "Principal Component Analysis": PCA(contamination = outliers_fraction,random_state=random_state)
                  }
                   
    for clf_name,clf in classifiers.items():
        t0 =time ()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, ndigits=4)
        time_list.append(duration)
                        
        roc = round(roc_auc_score(y_test,test_scores),ndigits = 4)
        prn = round(precision_n_scores(y_test,test_scores),ndigits = 4)
                        
        print("{clf_name} ROC:{roc}, precision @ rank n:{prn}, execution time: {duration}s".
                              format(clf_name=clf_name, roc =roc, prn = prn, duration = duration))
        roc_list.append(roc)
        prn_list.append(prn)
                        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df_columns = df_columns
    time_df = pd.concat([time_df,temp_df],axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df_columns = df_columns
    roc_df = pd.concat([roc_df,temp_df],axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df_columns = df_columns
    prn_df = pd.concat([prn_df,temp_df],axis=0)


... Processing arrhythmia.mat ...
Angle based outlier detector(ABOD) ROC:0.7246, precision @ rank n:0.2857, execution time: 4.4181s
Cluster-based Loacl Outlier Factor ROC:0.7283, precision @ rank n:0.2857, execution time: 2.7594s
Feature Bagging ROC:0.712, precision @ rank n:0.2857, execution time: 0.6101s
Histogram base outlier detection ROC:0.8077, precision @ rank n:0.4643, execution time: 1.7519s
Isolation Forest ROC:0.7972, precision @ rank n:0.4286, execution time: 0.5789s
k Nearest Neighbours (kNN) ROC:0.7148, precision @ rank n:0.2857, execution time: 0.0901s
Local Outlier Factor(LOF) ROC:0.7171, precision @ rank n:0.3214, execution time: 0.0743s


C:\Users\hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimun Covariance Determinant (MCD) ROC:0.725, precision @ rank n:0.3214, execution time: 1.7406s
One-Class SVM (OCSVM) ROC:0.7124, precision @ rank n:0.2857, execution time: 0.0859s
Principal Component Analysis ROC:0.7134, precision @ rank n:0.2857, execution time: 0.0741s

... Processing cardio.mat ...
Angle based outlier detector(ABOD) ROC:0.5928, precision @ rank n:0.275, execution time: 0.4035s
Cluster-based Loacl Outlier Factor ROC:0.7221, precision @ rank n:0.3, execution time: 0.1632s
Feature Bagging ROC:0.6003, precision @ rank n:0.2, execution time: 0.7619s
Histogram base outlier detection ROC:0.8556, precision @ rank n:0.525, execution time: 0.0101s
Isolation Forest ROC:0.9243, precision @ rank n:0.55, execution time: 0.3635s
k Nearest Neighbours (kNN) ROC:0.7663, precision @ rank n:0.3875, execution time: 0.1358s
Local Outlier Factor(LOF) ROC:0.6062, precision @ rank n:0.2, execution time: 0.0879s


C:\Users\hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.231552900163678 > -248.277052140188715). You may want to try with a higher value of support_fraction (current value: 0.510).
  RuntimeWarning)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-150.510611449116283 > -248.427240725426998). You may want to try with a higher value of support_fraction (current value: 0.510).
  RuntimeWarning)
C:\Users\hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Det

Minimun Covariance Determinant (MCD) ROC:0.8685, precision @ rank n:0.45, execution time: 0.5949s
One-Class SVM (OCSVM) ROC:0.9404, precision @ rank n:0.55, execution time: 0.07s
Principal Component Analysis ROC:0.9542, precision @ rank n:0.6375, execution time: 0.004s

... Processing glass.mat ...
Angle based outlier detector(ABOD) ROC:0.8571, precision @ rank n:0.2857, execution time: 0.0599s
Cluster-based Loacl Outlier Factor ROC:0.8373, precision @ rank n:0.1429, execution time: 0.0559s
Feature Bagging ROC:0.8571, precision @ rank n:0.1429, execution time: 0.0479s
Histogram base outlier detection ROC:0.7288, precision @ rank n:0.0, execution time: 0.004s
Isolation Forest ROC:0.745, precision @ rank n:0.1429, execution time: 0.3156s
k Nearest Neighbours (kNN) ROC:0.8427, precision @ rank n:0.1429, execution time: 0.008s
Local Outlier Factor(LOF) ROC:0.8499, precision @ rank n:0.1429, execution time: 0.004s
Minimun Covariance Determinant (MCD) ROC:0.7975, precision @ rank n:0.1429, e

C:\Users\hp\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
